### 生成数据集

In [5]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w =[2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype = torch.float)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size = labels.size()),dtype = torch.float)

### 读取数据

#### pytorch 提供了data 包来读取数据 由于data 常用作变量名 我们将导入的data模块用Data 代替 每一次迭代 我们将随机读取包含十个数据样本的小批量

In [8]:
import torch.utils.data as Data
batch_size = 10
dataset = Data.TensorDataset(features,labels)#将训练数据的特征和标签组合
#随机读取小批量
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.6618,  2.6453],
        [-0.2061,  0.9766],
        [ 0.0235, -0.5859],
        [ 0.7789,  0.4995],
        [ 0.0227, -0.8280],
        [ 0.3383, -1.1424],
        [ 2.3207, -0.0039],
        [ 0.0706,  0.2069],
        [-0.9952,  1.4954],
        [-1.4096, -2.0202]]) tensor([-3.4691,  0.4810,  6.2518,  4.0555,  7.0798,  8.7631,  8.8399,  3.6309,
        -2.8739,  8.2528])


### 定义模型

#### 首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用nn.Module实现一个线性回归模型。

In [31]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)
    # forword 定义前向传播
    def forward(self,x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
#可以使用nn.Sequential来更加方便的搭建网络，Sequential是一个有序的容器，网络层
#将按照在传入Sequential的顺序依次被添加到计算图中

In [36]:
#写法一
net = nn.Sequential(
    nn.Linear(num_inputs,1)
    #此处还可以传入其他层...
)
#写法二
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#net.add_module......

#写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
        ('linear',nn.Linear(num_inputs,1))
        #......
]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [ ]:
#可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器

In [37]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5418, -0.3849]], requires_grad=True)
Parameter containing:
tensor([-0.6385], requires_grad=True)


#### 作为一个单层神经网络，线性回归输出层中的神经单元和输入层中的各个输入完全连接。因此，线性回归的输出层又叫全连接层

#### 注意: torch.nn仅仅支持输入一个batch的样本不支持单个样本的输入，如果只有单个样本，可以使用 Input.unsqueeze(0)来添加一维。

### 初始化模型参数

In [ ]:
#pytorch 在init模块中提供了多种参数初始化的方法。这里的init是initializer的缩写
#我们通过init.normal_将权重参数每个元素初始化为随机采样于均值为0 标准差为0.01的
#正态分布 偏差会初始化为0

In [38]:
from torch.nn import init
init.normal_(net[0].weight,mean=0,std = 0.01)
init.constant_(net[0].bias, val=0)
# 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

### 定义损失函数

#### PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [27]:
loss = nn.MSELoss()

### 定义优化算法

In [28]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr = 0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


#### 我们还可以为不同子网络设置不同的学习率，这在funetune时经常用到 例：

In [40]:
optimizer = optim.SGD([
    {'params' : net[0].parameters()},
    #{'params' : net[1].parameters(),'lr':0.1}  
],lr = 0.03)

#### 有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

In [41]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

### 训练模型

#### 在使用Gluon训练模型时，我们通过调用optim实例的step函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。

In [42]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))


epoch 1, loss: 12.677607
epoch 2, loss: 3.737294
epoch 3, loss: 1.016642


#### 下面我们分别比较学到的模型参数和真实的模型参数。我们从net获得需要的层，并访问其权重（weight）和偏差（bias）。学到的参数和真实的参数很接近。

In [43]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.6411, -2.9523]], requires_grad=True)
4.2 Parameter containing:
tensor([3.5402], requires_grad=True)


#### 使用PyTorch可以更简洁地实现模型。torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的层，torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。
